In [61]:
import open3d as o3d
import numpy as np
from open3d import JVisualizer
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import copy 
import os

In [32]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    # orient normals so the ground is facing the same way
    pcd_down.orient_normals_to_align_with_direction()
    pcd_down = remove_ground_points(pcd_down, 0.4)

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [33]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds")
    source = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/fri_mar_12/corner.pcd")
    target = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/mon_mar_15_after_snow_2/corner.pcd")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [34]:
def remove_ground_points(pcd, z_threshold):
    
    pop_list = []
    for i, normal in enumerate(pcd.normals):
        if normal[2] > z_threshold:
            pop_list.append(i)
    
    print("Number of points to be removed: ", len(pop_list))
    pop_list.sort(reverse=True)
    for item in pop_list:
        pcd.normals.pop(item)
        pcd.points.pop(item)
    
    return pcd

In [35]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [36]:
voxel_size = 0.4
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.400.
:: Estimate normal with search radius 0.800.
Number of points to be removed:  14595
:: Compute FPFH feature with search radius 2.000.
:: Downsample with a voxel size 0.400.
:: Estimate normal with search radius 0.800.
Number of points to be removed:  19636
:: Compute FPFH feature with search radius 2.000.


In [37]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
         3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(10000000, 0.999))
    return result

In [38]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)


:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.400,
   we use a liberal distance threshold 0.600.
RegistrationResult with fitness=1.322645e-02, inlier_rmse=3.761809e-01, and correspondence_set size of 33
Access transformation to get result.


In [39]:
draw_registration_result(source, target, result_ransac.transformation)

In [40]:
print(result_ransac.transformation)

[[ 0.85081168 -0.49510347  0.17604555  0.05610032]
 [ 0.49052328  0.86846623  0.0717866  -0.81825822]
 [-0.18843141  0.02527756  0.98176099  0.31087464]
 [ 0.          0.          0.          1.        ]]


In [41]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    radius_normal = voxel_size * 2
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    target.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    source.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    # orient normals so the ground is facing the same way
    target.orient_normals_to_align_with_direction()
    source.orient_normals_to_align_with_direction()
    # remove all points with a z normal component greater than the threshold
    target = remove_ground_points(target, 0.3)
    source = remove_ground_points(source, 0.3)
    result = o3d.pipelines.registration.registration_icp(
        source_down, target_down, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

In [42]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.160.
Number of points to be removed:  27483
Number of points to be removed:  22978
RegistrationResult with fitness=1.439100e-01, inlier_rmse=1.185740e-01, and correspondence_set size of 2519
Access transformation to get result.


In [43]:
np.set_printoptions(suppress=True)
print(result_icp.transformation)

[[ 0.85167683 -0.49231304  0.17965092 -0.04157744]
 [ 0.48732402  0.87007429  0.07406775 -0.70046279]
 [-0.19277417  0.02446642  0.98093808  0.62305917]
 [ 0.          0.          0.          1.        ]]


In [44]:
draw_registration_result(source, target, result_icp.transformation)

In [56]:
before_snow = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/fri_mar_12/all_points.pcd")
after_snow = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/mon_mar_15_after_snow_2/all_points.pcd")

draw_registration_result(before_snow, after_snow, result_icp.transformation)

In [58]:
# apply calculated transformations and output the all points clouds for m3c2 distance calculation
before_snow = copy.deepcopy(before_snow).transform(result_icp.transformation)
o3d.io.write_point_cloud("before_aligned.pcd", before_snow)
o3d.io.write_point_cloud("after_aligned.pcd", after_snow)

True

In [59]:
# check to make sure our new files are transformed correctly with an identity matrix transform
before_aligned = o3d.io.read_point_cloud("before_aligned.pcd")
after_aligned = o3d.io.read_point_cloud("after_aligned.pcd")
draw_registration_result(before_aligned, after_aligned, np.identity(4))

In [64]:
# run cloudcompare in BASH command
# This assumes cloud compare was installed using the SNAPD package
os.system('cloudcompare.CloudCompare -O "after_aligned.pcd" -O "before_aligned.pcd" -M3C2 "m3c2_params.txt" -SAVE_CLOUDS')

0

In [103]:
print(len(dists))
print(len(after_snow.colors))

def sigmoid(number):
    z = 1/(1 + np.exp(-number))
    
    return z

med = np.median(dists)

color_list = []
for i in range(0, len(after_snow.colors)):
    red = sigmoid((dists[i] - med))
    blue = sigmoid((dists[i] - med) - red)
    green = 0
    color = np.asarray([red, green, blue])
    after_snow.colors[i] = color



26560000
26560000
